In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import *
from sklearn.metrics import *
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import *

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
#tf.config.list_physical_devices('GPU')

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6877710355414440864
xla_global_id: -1
]


In [4]:
#############
# data load
#############

# path = './data.csv'
path = ''
#data = pd.read_csv(path)

# target(y) column
target = '' 

# classification = 0, Regression = 1
is_regrssion = 1

#x = data.drop(target,axis=1)
#y = data[target]

##### test iris #####
from sklearn.datasets import load_iris 
data = load_iris()
x = data.data
y = data.target
##### test iris #####
#### test boston ####
from sklearn.datasets import load_boston
data = load_boston()
x = data.data
y = data.target
#### test boston ####
print('shape x, y')
print(x.shape,y.shape)

shape x, y
(506, 13) (506,)


c:\Users\sagvd17\Anaconda3\envs\jpytorch\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California h

In [5]:
len(x.shape)

2

In [148]:
#############
# data split
#############
test_size=.3
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=test_size,random_state=1)
print('train_size',len(x_train))
print('test_size',len(x_test))

train_size 354
test_size 152


In [149]:
#############
# pre_propessor
#############

# default
print('shape x,y ',x_train.shape,y_train.shape)
if is_regrssion == 0:
    print('y_train unique',np.unique(y_train))
if x.shape > 2:
    #x_shape=x_train.shape[1:-1]
    pass
else:
    x_shape = x_train.shape[-1]
y_shape = len(np.unique(y_train))
dense_act = 'linear'
loss_func = 'mse'
mets = ['mae']

if is_regrssion == 0:
    # if classification
    mets = ['acc']
    dense_act = 'softmax'
    
    if len(np.unique(y_train)) > 2:
        # when datset need one-hot-encoding
        y_train = to_categorical(y_train)
        #y_test = to_categorical(y_test)
        loss_func = 'categorical_crossentropy'

        print('y one-hot-encoded')
        print(' y unique',np.unique(y_train))
    else:
        loss_func = 'binary_crossentropy'


shape x,y  (354, 13) (354,)


In [150]:
###########
# Modeling
###########
# 1. clear session #
keras.backend.clear_session()
# 2. model set
model = keras.models.Sequential()

# epochs
epochs = 500

# 3. layer set 
model.add( keras.layers.Input(shape=x_shape) )
model.add( keras.layers.Dense(64,activation='relu') )
model.add( keras.layers.Dense(32,activation='relu') )
model.add( keras.layers.Dense(32,activation='relu') )
model.add( keras.layers.Dense(16,activation='relu') )
model.add( keras.layers.Dense(y_shape, activation=dense_act) )


# 4. compile model #
model.compile(loss=loss_func,metrics=mets,optimizer='adam')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                896       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 32)                1056      
                                                                 
 dense_3 (Dense)             (None, 16)                528       
                                                                 
 dense_4 (Dense)             (None, 196)               3332      
                                                                 
Total params: 7,892
Trainable params: 7,892
Non-trainable params: 0
_________________________________________________________________


In [151]:
model.fit(x_train,y_train,epochs=epochs,verbose=1)

Epoch 1/500
12/12 [==============================] - 1s 2ms/step - loss: 552.5679 - mae: 21.2477
Epoch 2/500
12/12 [==============================] - 0s 2ms/step - loss: 448.0391 - mae: 17.7488
Epoch 3/500
12/12 [==============================] - 0s 2ms/step - loss: 315.4824 - mae: 13.9285
Epoch 4/500
12/12 [==============================] - 0s 2ms/step - loss: 197.6788 - mae: 10.5872
Epoch 5/500
12/12 [==============================] - 0s 2ms/step - loss: 120.6938 - mae: 7.9675
Epoch 6/500
12/12 [==============================] - 0s 2ms/step - loss: 88.2636 - mae: 7.0015
Epoch 7/500
12/12 [==============================] - 0s 2ms/step - loss: 70.4717 - mae: 6.1338
Epoch 8/500
12/12 [==============================] - 0s 2ms/step - loss: 67.1353 - mae: 5.5650
Epoch 9/500
12/12 [==============================] - 0s 2ms/step - loss: 61.1071 - mae: 5.6967
Epoch 10/500
12/12 [==============================] - 0s 2ms/step - loss: 58.2040 - mae: 5.5563
Epoch 11/500
12/12 [====================

In [153]:
y_pred = model.predict(x_test)
y_preds = []
correct = 0
if is_regrssion == 0:
    for idx, pred in enumerate(y_pred):
        y_preds.append(np.argmax(pred))
        y_preds=np.array(y_preds).reshape(-1)
else:
    y_preds=np.array(np.around(y_pred))
#acc = correct/len(y_pred)
#print(f'acc:{acc*100}%')

print(y_preds,len(y_preds))
print(y_test,len(y_test))

5/5 [==============================] - 0s 1ms/step
[[32. 32. 32. ... 32. 32. 32.]
 [26. 26. 26. ... 26. 26. 26.]
 [19. 18. 19. ... 19. 19. 19.]
 ...
 [19. 18. 19. ... 19. 18. 19.]
 [30. 30. 30. ... 30. 30. 30.]
 [17. 17. 17. ... 17. 17. 17.]] 152
[28.2 23.9 16.6 22.  20.8 23.  27.9 14.5 21.5 22.6 23.7 31.2 19.3 19.4
 19.4 27.9 13.9 50.  24.1 14.6 16.2 15.6 23.8 25.  23.5  8.3 13.5 17.5
 43.1 11.5 24.1 18.5 50.  12.6 19.8 24.5 14.9 36.2 11.9 19.1 22.6 20.7
 30.1 13.3 14.6  8.4 50.  12.7 25.  18.6 29.8 22.2 28.7 23.8  8.1 22.2
  6.3 22.1 17.5 48.3 16.7 26.6  8.5 14.5 23.7 37.2 41.7 16.5 21.7 22.7
 23.  10.5 21.9 21.  20.4 21.8 50.  22.  23.3 37.3 18.  19.2 34.9 13.4
 22.9 22.5 13.  24.6 18.3 18.1 23.9 50.  13.6 22.9 10.9 18.9 22.4 22.9
 44.8 21.7 10.2 15.4 25.3 23.3  7.2 21.2 11.7 27.  29.6 26.5 43.5 23.6
 11.  33.4 36.  36.4 19.  20.2 34.9 50.  19.3 14.9 26.6 19.9 24.8 21.2
 23.9 20.6 23.1 28.  20.  23.1 25.   9.7 23.9 36.1 13.4 12.7 39.8 10.4
 20.6 17.8 19.5 23.7 28.5 24.3 23.8 19.1 28